In [2]:
# 檢查 GPU 是否可用
!nvidia-smi

# 確認 TensorFlow 版本
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Tue Dec 17 14:01:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# 從 GitHub 下載 Face Mask Detection 資料集
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

# 檢查資料夾結構
!ls Face-Mask-Detection/dataset


Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 4590, done.
remote: Total 4590 (delta 0), reused 0 (delta 0), pack-reused 4590 (from 1)
Receiving objects: 100% (4590/4590), 186.72 MiB | 16.13 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Updating files: 100% (4155/4155), done.
with_mask  without_mask


In [14]:
import os

with_mask_count = len(os.listdir('Face-Mask-Detection/dataset/with_mask'))
without_mask_count = len(os.listdir('Face-Mask-Detection/dataset/without_mask'))

print(f"帶口罩圖片數量: {with_mask_count}")
print(f"無口罩圖片數量: {without_mask_count}")


帶口罩圖片數量: 2165
無口罩圖片數量: 1930


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 資料夾路徑
data_dir = 'Face-Mask-Detection/dataset'

# 資料增強與標準化
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    validation_split=0.2
)

# 訓練集
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# 驗證集
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.


In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# 載入 VGG16 預訓練模型（去除頂層）
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 凍結 VGG16 底層權重

# 添加自定義分類層
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 分類：with_mask / without_mask
])

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 顯示模型架構
model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,137,986 (80.64 MB)

 Trainable params: 6,423,298 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [17]:
# 訓練模型
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


103/103 ━━━━━━━━━━━━━━━━━━━━ 72s 636ms/step - accuracy: 0.5819 - loss: 1.9488 - val_accuracy: 0.8875 - val_loss: 0.3354
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 78s 612ms/step - accuracy: 0.8006 - loss: 0.4110 - val_accuracy: 0.9364 - val_loss: 0.2704
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 81s 594ms/step - accuracy: 0.8330 - loss: 0.3573 - val_accuracy: 0.9523 - val_loss: 0.1755
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 83s 613ms/step - accuracy: 0.8288 - loss: 0.3388 - val_accuracy: 0.9377 - val_loss: 0.1850
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 90s 681ms/step - accuracy: 0.8420 - loss: 0.3339 - val_accuracy: 0.9535 - val_loss: 0.1417
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 66s 615ms/step - accuracy: 0.8575 - loss: 0.3172 - val_accuracy: 0.9389 - val_loss: 0.1408
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 81s 613ms/step - accuracy: 0.8438 - loss: 0.3338 - val_accuracy: 0.9487 - val_loss: 0.1618
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 66s 609ms/step - accuracy: 0.8660 - loss: 0.2749 - val

In [18]:
# 解凍 VGG16 的後 4 層
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

# 重新編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 微調模型
history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)



Epoch 1/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 86s 742ms/step - accuracy: 0.5237 - loss: 4.1034 - val_accuracy: 0.5281 - val_loss: 0.6920
Epoch 2/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 622ms/step - accuracy: 0.5215 - loss: 0.6924 - val_accuracy: 0.5281 - val_loss: 0.6918
Epoch 3/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 69s 631ms/step - accuracy: 0.5207 - loss: 0.6923 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 4/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 628ms/step - accuracy: 0.5262 - loss: 0.6918 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 5/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 67s 623ms/step - accuracy: 0.5219 - loss: 0.6922 - val_accuracy: 0.5281 - val_loss: 0.6916


In [21]:
import requests
from PIL import Image
import numpy as np
import io

# 測試圖片函數
def test_image(image_url, model, classes):
    try:
        response = requests.get(image_url)
        img = Image.open(io.BytesIO(response.content)).resize((224, 224))
        img_array = np.array(img) / 255.0
        img_array = img_array.reshape((1, 224, 224, 3))

        prediction = model.predict(img_array)
        predicted_class = classes[np.argmax(prediction)]
        print(f"Predicted Class: {predicted_class}")
    except Exception as e:
        print(f"Error: {e}")

# 測試範例圖片
classes = ["No Mask", "Mask"]
image_url = input("Enter image URL: ")
test_image(image_url, model, classes)


Enter image URL: https://na.cx/i/eqzQJYw.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 605ms/step
Predicted Class: No Mask


In [22]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# 預測驗證集
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

# 混淆矩陣與分類報告
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

print('Classification Report')
target_names = ['No Mask', 'Mask']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


26/26 ━━━━━━━━━━━━━━━━━━━━ 12s 412ms/step
Confusion Matrix
[[432   0]
 [386   0]]
Classification Report
              precision    recall  f1-score   support

     No Mask       0.53      1.00      0.69       432
        Mask       0.00      0.00      0.00       386

    accuracy                           0.53       818
   macro avg       0.26      0.50      0.35       818
weighted avg       0.28      0.53      0.37       818



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
# 解凍 VGG16 的更多層（例如後 10 層）
base_model.trainable = True
for layer in base_model.layers[:-10]:  # 保留前 10 層凍結
    layer.trainable = False

# 重新編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 重新訓練模型
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


103/103 ━━━━━━━━━━━━━━━━━━━━ 73s 648ms/step - accuracy: 0.5118 - loss: 0.6933 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 67s 613ms/step - accuracy: 0.5146 - loss: 0.6931 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 83s 631ms/step - accuracy: 0.5253 - loss: 0.6919 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 82s 630ms/step - accuracy: 0.5282 - loss: 0.6916 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 626ms/step - accuracy: 0.5375 - loss: 0.6905 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 82s 626ms/step - accuracy: 0.5222 - loss: 0.6923 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 82s 631ms/step - accuracy: 0.5212 - loss: 0.6924 - val_accuracy: 0.5281 - val_loss: 0.6916
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 84s 636ms/step - accuracy: 0.5311 - loss: 0.6913 - val

In [24]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.5,
    brightness_range=[0.3, 1.5],
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [25]:
# 設定類別權重
class_weights = {0: 1.0, 1: 2.0}  # No Mask: 1.0, Mask: 2.0

# 重新訓練模型時加入 class_weight 參數
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    class_weight=class_weights,
    epochs=10
)


Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 78s 640ms/step - accuracy: 0.5106 - loss: 1.0295 - val_accuracy: 0.4719 - val_loss: 0.7052
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 637ms/step - accuracy: 0.4760 - loss: 0.9797 - val_accuracy: 0.4719 - val_loss: 0.7220
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 69s 635ms/step - accuracy: 0.4774 - loss: 0.9667 - val_accuracy: 0.4719 - val_loss: 0.7339
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 628ms/step - accuracy: 0.4564 - loss: 0.9624 - val_accuracy: 0.4719 - val_loss: 0.7420
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 623ms/step - accuracy: 0.4620 - loss: 0.9618 - val_accuracy: 0.4719 - val_loss: 0.7461
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 621ms/step - accuracy: 0.4639 - loss: 0.9617 - val_accuracy: 0.4719 - val_loss: 0.7492
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 627ms/step - accuracy: 0.4846 - loss: 0.9593 - val_accuracy: 0.4719 - val_loss: 0.7500
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 68s 628ms/step - accuracy: 0.4693 - loss: 0

In [26]:
# 測試圖片預處理一致性
def test_image(image_url, model, classes):
    try:
        response = requests.get(image_url)
        img = Image.open(io.BytesIO(response.content)).convert('RGB').resize((224, 224))
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        prediction = model.predict(img_array)
        predicted_class = classes[np.argmax(prediction)]
        print(f"Predicted Class: {predicted_class}")
    except Exception as e:
        print(f"Error: {e}")


In [27]:
from sklearn.metrics import confusion_matrix, classification_report

# 預測驗證集
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

# 混淆矩陣和分類報告
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

print('Classification Report')
target_names = ['No Mask', 'Mask']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 541ms/step
Confusion Matrix
[[  0 432]
 [  0 386]]
Classification Report
              precision    recall  f1-score   support

     No Mask       0.00      0.00      0.00       432
        Mask       0.47      1.00      0.64       386

    accuracy                           0.47       818
   macro avg       0.24      0.50      0.32       818
weighted avg       0.22      0.47      0.30       818



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
import requests
from PIL import Image
import numpy as np
import io

# 測試圖片函數
def test_image(image_url, model, classes):
    try:
        response = requests.get(image_url)
        img = Image.open(io.BytesIO(response.content)).resize((224, 224))
        img_array = np.array(img) / 255.0
        img_array = img_array.reshape((1, 224, 224, 3))

        prediction = model.predict(img_array)
        predicted_class = classes[np.argmax(prediction)]
        print(f"Predicted Class: {predicted_class}")
    except Exception as e:
        print(f"Error: {e}")

# 測試範例圖片
classes = ["No Mask", "Mask"]
image_url = input("Enter image URL: ")
test_image(image_url, model, classes)


Enter image URL: https://na.cx/i/eqzQJYw.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
Predicted Class: Mask
